Hi all, this is some of the work I did previously when looking at driver behavior as it relates to crashes/injuries. Much of this work could be updated to include more recent years: 2020, 2021, and 2022. Of course, this will come with new issues in the data analysis given the effects of the pandemic. 

This is based on our internal Python library and datasets we use in our database--but these (or very similar versions) should also exist for you all to access given our data-sharing for this project. If something does not make sense, Seth should be able to troubleshoot. I am also available in a limited way for follow up. These are just some ideas for analysis and draft code--feel free to disregard. 

Best, 
Chris

## General Notes

Analysis:
- We no longer receive all PDO (Propery Damage Only) crashes as of 2020 from NYPD. Any analysis using 2020-onwards data could compare C injuries crashes (or C and B injuries crashes) versus severe (A) injuries crashes versus fatality crashes to get a better idea of severity. 
- This includes road network (intersection/block) crashes as well as highway crashes. These can be separated as they are somewhat different in nature. Furthermore, speed cameras only exist on the local level, so there is a different sample (potentially) when thinking about people getting in highway crashes (more out of state?) versus local crashes and how that interacts with the violation data.

Data Notes: 
- Where I use 'working.speed_camera_vio' -- this is just the DOF Speed Camera/Parking Violations filtered on only **PHTO SCHOOL ZN SPEED VIOLATION**
- Where I use 'working.rlc_vio' -- this is just the DOF Speed Camera/Parking Violations filtered on only **FAILURE TO STOP AT RED LIGHT**
- Where I use 'working.vio_all' (deprecated) -- this is the combination of these two

## 0.0 Setup

In [ ]:
writer = pd.ExcelWriter('BehaviorStudyInitialResults_20230405.xlsx')
db = pysqldb.DbConnect(type='pg',server='dotdevrhpgsql01', database='ris', allow_temp_tables=True, quiet=True)

In [ ]:
db = pysqldb.DbConnect(type='pg',server='dotdevrhpgsql01', database='ris', allow_temp_tables=True, quiet=True)

In [ ]:
fatal_sql = pysqldb.DbConnect(type='MS', server='dotgissql01', database='Fatality', ldap=True, quiet=True)
forms_sql = pysqldb.DbConnect(type='MS', server='dot55sql01', database='FORMS', user='arcgis', password='arcgis', quiet=True)

# 1.0 Standardized Cleaned Tables

Returns non-junk plates

Context: there are many plates in the data that are just junk. To try to limit studies to valid plates/licenses, I attempted to remove these.

In [ ]:
if not db.table_exists(schema='working', table='cleaned_plates'):
    db.dfquery("""

    drop table if exists working.cleaned_plates; 
    create table working.cleaned_plates as 

    select distinct  
        lower(plate_num) as plate_num
    from 
        wc_accident_vehicle_f 
    where
        plate_num is not null 
        and lower(plate_num) not in (
            'ladder 1', 'fire', 'usps', 'police', 'uknown', 'not regi', 'bicycle', 'fdny', 
            'unk', 'unkn', 'unkown', 'unknown', 'none', 'n/a', 'n\\a', 'no plate', 'na', 
            'zzzzzr1', 'zzzzzzum', 'zzz-zzzz', '`', '|||', '|()|', '__', '?', '? ? ?', 
            '????', '?????', '??????', '???????', '????????', 
            '///////', '////////', '@*$$|$', '@%^$', '*', '*****', '*******', '', 
            '\\\\\\\\\\\\\\\\', '&^*@', '+.', '00000.', '000-000-', '00000000', '00000001', '0000000o', 
            '0000001', '999999999', '000000000', 'na', '99999999', '123456789', '00000000', '9999999999', 
            '0000000000', '9999999', '000000', '999999', 'exempt', '111111111', 'unk', '0000000', 'none', 
            '00000000000', 'unknown', '99999999999', '0', '0000', 'pd', '000000000000', '99999', '999999999999', 
            '00000', '0000000000000', '000', '.', 'p', '999', '9999', '00000000000000', '00000000000000000000',
            '178084863', 'e57372256110582', 'xxxxxxxxx', '000000001', 'onfile', '99999999999999', '890463639', 
            '000000000000000', 'unlicensed', 'policeinvolved', 's', '2', 'left sce', 'engine 2',  'noplate', 'ukn', 
            'not appl', 'bike', 't')
        and not (length(plate_num) = 4 and plate_num like '20%')
        and replace(plate_num, '0', '') != ''
        and replace(plate_num, '1', '') != ''
        and replace(plate_num, '9', '') != ''
        and replace(plate_num, 'x', '') != ''
        and replace(plate_num, '-', '') != ''
        and replace(plate_num, '.', '') != ''
        and replace(lower(plate_num), 'z', '') != ''
        and replace(lower(plate_num), ' ', '') != '\\'
        and plate_num not like '%unregist%'
        and plate_num not like '%temp%'
        and crash_date >= '2017-01-01'

    """)

Returns non-junk license numbers

In [ ]:
if not db.table_exists(schema='working', table='cleaned_licenses'):

    db.dfquery("""
    drop table if exists working.cleaned_licenses; 
    create table working.cleaned_licenses as 

    select distinct  
        driver_license_num
    from 
        wc_accident_vehicle_f 
    where
        driver_license_num is not null 

        and lower(driver_license_num) not in (
            'ladder 1', 'fire', 'usps', 'police', 'uknown', 'not regi', 'bicycle', 'fdny', 
            'unk', 'unkn', 'unkown', 'unknown', 'none', 'n/a', 'n\\a', 'no plate', 'na', 
            'zzzzzr1', 'zzzzzzum', 'zzz-zzzz', '`', '|||', '|()|', '__', '?', '? ? ?', 
            '????', '?????', '??????', '???????', '????????', 
            '///////', '////////', '@*$$|$', '@%^$', '*', '*****', '*******', '', 
            '\\\\\\\\\\\\\\\\', '&^*@', '+.', '00000.', '000-000-', '00000000', '00000001', '0000000o', 
            '0000001', '999999999', '000000000', 'na', '99999999', '123456789', '00000000', '9999999999', 
            '0000000000', '9999999', '000000', '999999', 'exempt', '111111111', 'unk', '0000000', 'none', 
            '00000000000', 'unknown', '99999999999', '0', '0000', 'pd', '000000000000', '99999', '999999999999', 
            '00000', '0000000000000', '000', '.', 'p', '999', '9999', '00000000000000', '00000000000000000000',
            '178084863', 'e57372256110582','000000001', 'onfile', '99999999999999', '890463639', 
            '000000000000000', 'unlicensed', 'policeinvolved', 's', '2', 'left sce', 'engine 2',  'noplate', 'ukn', 'fd', 
            'rked', 'nolicense')

        and replace(driver_license_num, '1', '') != ''
        and replace(driver_license_num, '0', '') != ''
        and replace(driver_license_num, '9', '') != ''
        and replace(driver_license_num, 'x', '') != ''
        and replace(driver_license_num, '-', '') != ''
        and replace(driver_license_num, '.', '') != ''
        and replace(driver_license_num, '*', '') != ''
        and replace(lower(driver_license_num), 'z', '') != ''
        and replace(lower(driver_license_num), ' ', '') != '\\'
        and driver_license_num not like '%unregist%'
        and driver_license_num not like '%temp%'
        and crash_date >= '2017-01-01'

    """)

Returns if a plate has a TLC pattern

TLC indicators based off of:

- TC/YC 
- Dollar Van plates 
- Number/letter/number/number/letter
- Number/letter/number/number http://www.nyc.gov/html/tlc_medallion_info/html/tlc_lookup.shtml
- Letter/letter/number/number/number
- Letter/letter/letter/number/number/number

In [ ]:
if not db.table_exists('tlc_plate_ind', schema='working'):

    db.dfquery("""

    drop table if exists working.tlc_plate_ind; 
    create table working.tlc_plate_ind as 

    select 
        replace(lower(plate_num), ' ', '') as plate_num,
        case 
            when lower(plate_num) like 't%c' then True 
            when lower(plate_num) like 'y%c' then True 
            when lower(plate_num) like 'o%l' then True 
            when 
                lower(plate_num) like '%lv' 
                or lower(plate_num) like '%lv' 
                or lower(plate_num) like '%lb' 
                or lower(plate_num) like '%la' 
                or lower(plate_num) like '%bb' 
                then True 
            when lower(plate_num) ~* '^([0-9]){1}([a-z]){1}([0-9]){1}([0-9]){1}([a-z]){1}$' then True
            when lower(plate_num) ~* '^([0-9]){1}([a-z]){1}([0-9]){1}([0-9]){1}$' then True 
            when lower(plate_num) ~* '^([a-z]){1}([a-z]){1}([0-9]){1}([0-9]){1}([0-9]){1}$' then True
            when lower(plate_num) ~* '^([a-z]){1}([a-z]){1}([a-z]){1}([0-9]){1}([0-9]){1}([0-9]){1}$' then True
            else False 
        end as tlc_veh 
    from 
        wc_accident_vehicle_f 

    """)

**Note: the data has a TLC-involved indicator. The license plate is another attempt to identify for-hire vehicles. I thought this could be an important differentiator in driver behavior, and I recommend exploring 'normal' driver and TLC behavior separately, if possible, or at least exploring that possibility.**

Returns single driver plates 

A plate can have at most one license number associated with it (and nulls)

In [ ]:
if not db.table_exists('single_driver_plates', schema='working'):
    db.dfquery("""

    drop table if exists working.single_driver_plates; 
    create table working.single_driver_plates as 

    select 
        replace(lower(ve.plate_num), ' ', '') as plate_num
    from 
        wc_accident_vehicle_f ve
    
    join 
        working.cleaned_plates cp  
    on 
        lower(ve.plate_num)=cp.plate_num 
    
    join 
        working.cleaned_licenses cl 
    on 
        ve.driver_license_num=cl.driver_license_num
    
    where 
        ve.crash_date >= '2017-01-01'
    
    group by 1 
    
    having count(distinct ve.driver_license_num) < 2

    """)

#### Driver Demographics 
 - The ris cleaned version of wc_accident_victim_f doesn't have the non-injured parties (for ex., a non-injured driver) 
 - Manually pulling from SQL Server

In [ ]:
# db.drop_table('working', 'driver_demo')

if not db.table_exists(schema='working', table='driver_demo'):
    q = """

    select 
        ve.accident_id as crashid,
        plate_num, 
        cast(norm_driver_license_number as nvarchar(max)) as driver_license_num,
        VICTIM_AGE, 
        VICTIM_SEX
    from 
        wc_accident_vehicle_f ve 

    left join 
        wc_accident_victim_f vi

    on
        ve.ACCIDENT_ID=vi.ACCIDENT_ID 
        and ve.vehicle_num=vi.vehicle_num 
        and vi.person_role_code='Driver' 

    where
        ve.ACCIDENT_DT >= '2017-01-01'
        and ve.ACCIDENT_DT <= '2021-12-31'

    """

    pysqldb.sql_to_pg_qry(forms_sql, db, q, dest_schema='working', dest_table='driver_demo') #, encoding='LATIN1')

## 2.0 Contributing Factors Yearly Breakdown for Injury Crashes

To Do:
- Add % diff of category, not just % of all crashes to breakdowns 

In [ ]:
all_inj_cf = db.dfquery("""

with cf_breakdown as (
    
    select  
        case 
            when crash_date between '2017-01-01' and '2017-12-31' then 2017
            when crash_date between '2018-01-01' and '2018-12-31' then 2018 
            when crash_date between '2019-01-01' and '2019-12-31' then 2019 
        end as period, 
        initcap(cf) as cf, 
        count(distinct crashid)
    from (
        select 
            --GET DISTINCT PERIOD/CF COUNT FOR ALL INJ CRASH 
            ve.contributing_factor_1 as cf, 
            ve.crashid, 
            ve.crash_date
        from
            wc_accident_vehicle_f ve
        join 
            (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
        on 
            i.crashid=ve.crashid
        where 
            crash_date between '2017-01-01' and '2019-12-31'

        union 

        select 
            --GET DISTINCT PERIOD/CF COUNT FOR ALL INJ CRASH 
            ve.contributing_factor_2 as cf, 
            ve.crashid, 
            ve.crash_date
        from
            wc_accident_vehicle_f ve
        join 
            (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
        on 
            i.crashid=ve.crashid
        where 
            crash_date between '2017-01-01' and '2019-12-31'
    ) c
    group by 1, 2
), 

period_breakdown as (
    --GET DISTINCT PERIOD COUNT FOR ALL INJ CRASH 
    select 
        case 
            when crash_date between '2017-01-01' and '2017-12-31' then 2017
            when crash_date between '2018-01-01' and '2018-12-31' then 2018 
            when crash_date between '2019-01-01' and '2019-12-31' then 2019 
        end as period, 
        count(distinct a.crashid) as total
    from 
        wc_accident_f a 
    join 
        (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
    on 
        i.crashid=a.crashid
    where 
        crash_date between '2017-01-01' and '2019-12-31'
    group by 1
)

select 
    cf.*, t.total
from 
    cf_breakdown cf
right join 
    period_breakdown t
on 
    cf.period=t.period

""")

# Calculate fraction of period's crashes for each contributing factor 
all_inj_cf['frac'] = all_inj_cf['count']/all_inj_cf['total']*100
all_inj_cf = all_inj_cf.pivot(index='cf', columns='period', values='frac').reset_index()

# Filter to those that are always > 0.5% of crashes (> 1/200)
cols_df = all_inj_cf.set_index('cf').T.min().reset_index()
cols = list(cols_df[(cols_df[0] > 0.5) & (~cols_df['cf'].isna())]['cf'])

# Aesthetic
all_inj_cf = all_inj_cf[all_inj_cf['cf'].isin(cols)]
all_inj_cf = all_inj_cf.reset_index(drop=True).rename(columns={
        'cf': 'Contributing Factor',
}).round(2)

all_inj_cf.sort_values(by=2019, ascending=False).to_excel(writer, 'ContributingFactorsYOY', index=False)

### 2.1 Contributing Factors for Repeat Vehicles 

Analysis Idea: look at the contributing factor breakdown for severe injuries and/or fatality crashes and compare those breakdowns to the 'normal' crashes (C injuries only). I attempted to do this here by using repeat vehicles in the crash data as a 'high risk' group.

In [ ]:
all_inj_cf = db.dfquery("""

with multi_plates as (
    
    select 
        distinct crashid 
    from 
        wc_accident_vehicle_f ve 
    
    join 
        working.cleaned_plates cp 
    on 
        lower(ve.plate_num)=cp.plate_num 
        
    where 
       replace(lower(ve.plate_num), ' ', '') in ( 
    
        select 
            replace(lower(ve.plate_num), ' ', '') plate_num

        from 
            wc_accident_vehicle_f ve 

        join 
            working.cleaned_plates cp 
        on 
            lower(ve.plate_num)=cp.plate_num 
        
        where 
            ve.crash_date >= '2017-01-01'

        group by 1 

        having count(distinct ve.crashid) > 1 
    
    ) 
), 

cf_breakdown as (
    
    select  
        case 
            when crash_date between '2017-01-01' and '2017-12-31' then 2017
            when crash_date between '2018-01-01' and '2018-12-31' then 2018 
            when crash_date between '2019-01-01' and '2019-12-31' then 2019 
        end as period, 
        cf, 
        count(distinct crashid)
    from (
        select 
            --GET DISTINCT PERIOD/CF COUNT FOR ALL INJ CRASH 
            initcap(ve.contributing_factor_1) as cf, 
            ve.crashid, 
            ve.crash_date
        from
            wc_accident_vehicle_f ve
        join 
            (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
        on 
            i.crashid=ve.crashid
        join 
            wc_accident_f a 
        on 
            ve.crashid=a.crashid
        join 
            multi_plates mp 
        on 
            mp.crashid=a.crashid
        where 
            ve.crash_date between '2017-01-01' and '2019-12-31'

        union 

        select 
            --GET DISTINCT PERIOD/CF COUNT FOR ALL INJ CRASH 
            initcap(ve.contributing_factor_2) as cf, 
            ve.crashid, 
            ve.crash_date
        from
            wc_accident_vehicle_f ve
        join 
            (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
        on 
            i.crashid=ve.crashid
        join 
            wc_accident_f a 
        on 
            ve.crashid=a.crashid
        join 
            multi_plates mp 
        on 
            mp.crashid=a.crashid
        where 
            ve.crash_date between '2017-01-01' and '2019-12-31'
    ) c
    group by 1, 2
), 

period_breakdown as (
    --GET DISTINCT PERIOD COUNT FOR ALL INJ CRASH 
    select 
        case 
            when ve.crash_date between '2017-01-01' and '2017-12-31' then 2017
            when ve.crash_date between '2018-01-01' and '2018-12-31' then 2018 
            when ve.crash_date between '2019-01-01' and '2019-12-31' then 2019 
        end as period, 
        count(distinct a.crashid) as total
    from 
        wc_accident_f a 
    join 
        (select distinct crashid from wc_accident_victim_f where inj_killed='Injured') i 
    on 
        i.crashid=a.crashid
    join 
        wc_accident_vehicle_f ve 
    on 
        ve.crashid=a.crashid
    join 
        multi_plates mp 
    on 
        mp.crashid=a.crashid
    where 
        ve.crash_date between '2017-01-01' and '2019-12-31'
    group by 1
)

select 
    cf.*, t.total
from 
    cf_breakdown cf
right join 
    period_breakdown t
on 
    cf.period=t.period

""")

# Calculate fraction of period's crashes for each contributing factor 
all_inj_cf['frac'] = all_inj_cf['count']/all_inj_cf['total']*100
all_inj_cf = all_inj_cf.pivot(index='cf', columns='period', values='frac').reset_index()

# Filter to those that are always > 0.5% of crashes (> 1/200)
cols_df = all_inj_cf.set_index('cf').T.min().reset_index()
cols = list(cols_df[(cols_df[0] > 0.5) & (~cols_df['cf'].isna()) & (cols_df['cf'] != 'unspecified')]['cf'])

# Aesthetic
all_inj_cf = all_inj_cf[all_inj_cf['cf'].isin(cols)]
all_inj_cf = all_inj_cf.reset_index(drop=True).rename(columns={
        'cf': 'Contributing Factor',
}).round(2)

all_inj_cf.sort_values(by=2019, ascending=False).to_excel(writer, 'ContributingFactorsRepeat', index=False)

## 3.0 Targeted Narrative Keywords in FORMS

This was an attempt to see if the narratives were indicating an increase in a variety of phenomena. I would ignore this.

In [ ]:
db.dfquery(""" 

select 
    date_part('year', crash_date)::int as year, 
    case 
        when f.forms_crashid is not null then 'Fatal'
        when i.crashid is not null then 'Injury' 
        else 'PDO' 
    end as status,
    100*count(distinct case when narrative like '% drug%' then a.crashid end)::float/count(*) as drug , 
    100*count(distinct case when narrative like '% alcohol%' then a.crashid end)::float/count(*) as alcohol, 
    100*count(distinct case when narrative like '% bac %' then a.crashid end)::float/count(*) as bac , 
    100*count(distinct case when narrative like '% drunk%' then a.crashid end)::float/count(*) as drunk , 
    100*count(distinct case when narrative like '% speed%' then a.crashid end)::float/count(*) as speed , 
    100*count(distinct case when narrative like '% reckless%' then a.crashid end)::float/count(*) as reckless , 
    100*count(distinct case when narrative like '% phone%' then a.crashid end)::float/count(*) as phone , 
    100*count(distinct case when narrative like '% medication%' then a.crashid end)::float/count(*) as medication , 
    100*count(distinct case when narrative like '% unsafe%' then a.crashid end)::float/count(*) as unsafe, 
    100*count(distinct case when narrative like '% dangerous%' then a.crashid end)::float/count(*) as dangerous , 
    100*count(distinct case when narrative like '% distract%' then a.crashid end)::float/count(*) as distract, 
    100*count(distinct case when narrative like '% sleep%' then a.crashid end)::float/count(*) as sleep, 
    100*count(distinct case when narrative like '% inexperience%' then a.crashid end)::float/count(*) as inexperience, 
    100*count(distinct case when narrative like '% improper%' then a.crashid end)::float/count(*) as improper , 
    100*count(distinct case when narrative like '% disregard%' then a.crashid end)::float/count(*) as disregard, 
    100*count(distinct case when narrative like '% illness%' then a.crashid end)::float/count(*) as illness, 
    100*count(distinct case when narrative like '% illegal%' then a.crashid end)::float/count(*) as illegal, 
    100*count(distinct case when narrative like '% headphone%' then a.crashid end)::float/count(*) as headphone, 
    100*count(distinct case when narrative like '% inattention%' then a.crashid end)::float/count(*) as inattention, 
    count(*)
from 
    wc_accident_f a 
left join 
    (select distinct crashid from wc_accident_victim_f) i 
on 
    a.crashid=i.crashid 
left join 
    (select distinct forms_crashid from fatal_crash) f 
on 
    a.crashid=f.forms_crashid::numeric
where 
    crash_date between '2017-01-01' and '2019-12-31'
group by 1, 2 
order by 2, 1 

""").round(2).to_excel(writer, 'TargetedKeywordsByCrashType', index=False)

## 4.0 Violations Numbers 

This section simply gets violation counts by year and type of license plate (commercial, passenger, etc.). 

Reference codes found here: https://dmv.ny.gov/registration/registration-class-codes

In [ ]:
db.dfquery("""

select 
    date_part('year', issue_date::date)::int as year, 
    license_type, 
    count(*) 
from 
    working.speed_camera_vio 
group by 
    1, 2

""").pivot(index='license_type', columns='year', values='count').sort_values(by=2021, ascending=False).reset_index().to_excel(writer, 'AllPlateViolationsByYearByVehicleType', index=False)

Looking at NYS plates only

In [ ]:
db.dfquery("""

select 
    date_part('year', issue_date::date)::int as year, 
    license_type, 
    count(*) 
from 
    working.speed_camera_vio 
where 
    state='NY'
group by 
    1, 2

""").pivot(index='license_type', columns='year', values='count').sort_values(by=2021, ascending=False).reset_index().to_excel(writer, 'NYPlateViolationsByYearByVehicleType', index=False)

Red light camera yearly counts

In [ ]:
db.dfquery("""

select 
    date_part('year', issue_date::date)::int as year, 
    license_type, 
    count(*) 

from 
    working.rlc_vio

group by 
    1, 2

""").pivot(index='license_type', columns='year', values='count').sort_values(by=2021, ascending=False).reset_index().to_excel(writer, 'AllPlateRLCViolationsByYearByVehicleType', index=False)

RLC for NY Only

In [ ]:
db.dfquery("""

select 
    date_part('year', issue_date::date)::int as year, 
    license_type, 
    count(*) 

from 
    working.rlc_vio

where 
    state='NY'

group by 
    1, 2

""").pivot(index='license_type', columns='year', values='count').sort_values(by=2021, ascending=False).reset_index().to_excel(writer, 'NYPlateRLCViolationsByYearByVehicleType', index=False)

## 5.0 Violations per Plate 

Looking at types with a non-negligible number of violations for NY plates

In [ ]:
relevant_types = set(db.dfquery("""

select 
    license_type, 
    count(*)
from
    working.speed_camera_vio 
where
    state='NY'
group by 
    1
having count(*) > 10000 
order by 2 desc
""")['license_type'])

Repeating for RLC

In [ ]:
relevant_types_rlc = set(db.dfquery("""

select 
    license_type, 
    count(*)
from
    working.rlc_vio
where
    state='NY'
group by 
    1
having count(*) > 10000 
order by 2 desc
""")['license_type'])

Looking at the breakdown of the number of violations by plate by plate type for NY plates only

In [ ]:
breakdown = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.speed_camera_vio 

    where 
        state='NY'

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 
""")

breakdown[breakdown['license_type'].isin(relevant_types)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.reset_index().to_excel(writer, 'NYPlatesViolationsOccurrenceAllTime', index=False)

Breakdown above for RLC only

In [ ]:
breakdown_rlc = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.rlc_vio

    where 
        state='NY'

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 
""")

breakdown_rlc[breakdown_rlc['license_type'].isin(relevant_types_rlc)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.reset_index().to_excel(writer, 'NYPlatesViolationsOccurrenceAllTimeRLC', index=False)

The Above Repeated for 2020, 2019 Only

In [ ]:
breakdown = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.speed_camera_vio 

    where 
        state='NY'
        and date_part('year', issue_date::date)::int = 2020

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 

""")

breakdown[breakdown['license_type'].isin(relevant_types)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.reset_index().to_excel(writer, 'NYPlatesViolationsOccurrence2020', index=False)

breakdown = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.speed_camera_vio 

    where 
        state='NY'
        and date_part('year', issue_date::date)::int = 2019

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 

""")

breakdown[breakdown['license_type'].isin(relevant_types)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.reset_index().to_excel(writer, 'NYPlatesViolationsOccurrence2019', index=False)

In [ ]:
# 2020 breakdown RLC
breakdown = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.rlc_vio

    where 
        state='NY'
        and date_part('year', issue_date::date)::int = 2020

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 

""")

breakdown[breakdown['license_type'].isin(relevant_types)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.to_excel(writer, 'NYPlatesViolationsOccurrenceRLC2020', index=False)

In [ ]:
# 2019 breakdown RLC
breakdown = db.dfquery("""

select 
    viol_count, 
    license_type, 
    count(*)
from (
    select 
        license_type,
        plate, 
        count(*) as viol_count 

    from 
        working.rlc_vio

    where 
        state='NY'
        and date_part('year', issue_date::date)::int = 2019

    group by 
        1, 2
) pc 
group by 1, 2
order by 2, 1 asc 

""")

breakdown[breakdown['license_type'].isin(relevant_types)].pivot(index='license_type', columns='viol_count', values='count').fillna(0).T.to_excel(writer, 'NYPlatesViolationsOccurrenceRLC2019', index=False)

## 6.0 Violation Involvement across Crash Types 

For a given vehicle in a crash (vehicle_crash), gets the previous number of violations 

Notes: 
- Right now, limits time-frame to 2019 only (**should update or try other timeframes**)
- To explore: highway vs. non-highway breakdown
- To explore: number of violations in a given time frame (1Y prior crash, 2Y, etc.)
- To explore: below limits to only vehicle type car/suv but this could be removed or changed

In [ ]:
prev_vio = db.dfquery("""

select distinct 
    concat(ve.vehicle_num, '-', ve.crashid) as vehicle_crash,
    case 
        when fl.forms_crashid is not null then 'Fatal'
        when inj.crashid is not null then 'Injury'
        else 'PDO' 
    end as type,
    count(distinct v_all.*) as vio_num_prior
from 
    wc_accident_vehicle_f ve 

join
    wc_accident_f a 
on 
    ve.crashid=a.crashid 
    --and a.loc in ('mid', 'int')

--JOIN CRASHES TO CLEANED PLATES TO LIMIT 
join 
    working.cleaned_plates cp 
on 
    lower(ve.plate_num)=cp.plate_num 

--LIMIT TO NON-TLC ONLY 
join 
    working.tlc_plate_ind tlc 
on 
    lower(ve.plate_num)=tlc.plate_num 
    and tlc.tlc_veh='False' 

left join 
    (select distinct forms_crashid from fatal_crash) fl
on 
    ve.crashid=fl.forms_crashid::numeric

left join 
    (select distinct crashid from wc_accident_victim_f where inj_killed='injured') inj
on 
    ve.crashid=inj.crashid

left join 
    (select distinct plate, issue_date from working.speed_camera_vio) v_all
on 
    lower(ve.plate_num)=lower(v_all.plate)
    and issue_date::date < ve.crash_date::date

where 
    date_part('year', ve.crash_date) = 2019
    and plate_state='NY'
    and veh_type_general='car/suv'

group by 1, 2

""")

Next Steps: 
- Update data 
- Remove PDO, switch to C/B Injury, A Injury, Fatality or C/B Injury vs KSI
- Explore timeframes

Gets those with 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10+ prior violations %

In [ ]:
# Pandas code to get the number of vehicle-crash incidences where the number of prior violations was 0, 1, ...10+
# Broken out by PDO, Injury, Fatal 

data = {}
for typ in ('PDO', 'Injury', 'Fatal'):
    data[typ] = {}
    for i in range(1, 11):
        tmp_df = prev_vio[prev_vio['type'] == typ]
        if i == 10:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] >= i])/(len(tmp_df)*1.0)
        else:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] == i])/(len(tmp_df)*1.0)

pd.DataFrame(data).reset_index().to_excel(writer, 'PastViolationsByCrashType10+', index=False)


Gets those with 0, 1, 2+ prior violations %

In [ ]:
# Pandas code to get the number of vehicle-crash incidences where the number of prior violations was 0, 1, 2+
# Broken out by PDO, Injury, Fatal 

data = {}
for typ in ('PDO', 'Injury', 'Fatal'):
    data[typ] = {}
    for i in range(1, 3):
        tmp_df = prev_vio[prev_vio['type'] == typ]
        if i == 2:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] >= i])/(len(tmp_df)*1.0)
        else:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] == i])/(len(tmp_df)*1.0)

pd.DataFrame(data).reset_index().to_excel(writer, 'PastViolationsByCrashType2+', index=False)

Same as above for Red Light Camera

In [ ]:
prev_vio = db.dfquery("""

select distinct 
    concat(ve.vehicle_num, '-', ve.crashid) as vehicle_crash,
    case 
        when fl.forms_crashid is not null then 'Fatal'
        when inj.crashid is not null then 'Injury'
        else 'PDO' 
    end as type,
    count(distinct v_all.*) as vio_num_prior 
from 
    wc_accident_vehicle_f ve 

join
    wc_accident_f a 
on 
    ve.crashid=a.crashid 
    and a.loc in ('mid', 'int')

join 
    working.cleaned_plates cp 
on 
    lower(ve.plate_num)=cp.plate_num 


join 
    working.tlc_plate_ind tlc 
on 
    lower(ve.plate_num)=tlc.plate_num 
    and tlc.tlc_veh='False' 

left join 
    (select distinct forms_crashid from fatal_crash) fl
on 
    ve.crashid=fl.forms_crashid::numeric

left join 
    (select distinct crashid from wc_accident_victim_f where inj_killed='injured') inj
on 
    ve.crashid=inj.crashid

left join 
    (select distinct plate, issue_date from working.rlc_vio) v_all
on 
    lower(ve.plate_num)=lower(v_all.plate)
    and issue_date::date < ve.crash_date::date

where 
    date_part('year', ve.crash_date) = 2019
    and plate_state='NY'
    and veh_type_general='car/suv'

group by 1, 2

""")

In [ ]:
data = {}
for typ in ('PDO', 'Injury', 'Fatal'):
    data[typ] = {}
    for i in range(1, 11):
        tmp_df = prev_vio[prev_vio['type'] == typ]
        if i == 10:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] >= i])/(len(tmp_df)*1.0)
        else:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] == i])/(len(tmp_df)*1.0)

pd.DataFrame(data).reset_index().to_excel(writer, 'RLCPastViolationsByCrashType10+', index=False)

In [ ]:
data = {}
for typ in ('PDO', 'Injury', 'Fatal'):
    data[typ] = {}
    for i in range(1, 3):
        tmp_df = prev_vio[prev_vio['type'] == typ]
        if i == 2:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] >= i])/(len(tmp_df)*1.0)
        else:
            data[typ][i] = 100*len(tmp_df[tmp_df['vio_num_prior'] == i])/(len(tmp_df)*1.0)

pd.DataFrame(data).reset_index().to_excel(writer, 'RLCPastViolationsByCrashType2+', index=False)

## 7.0 Violation Crash Correlation 

I tried to do a correlation between the number of crashes and the number of violations. This did not work out because crashes are such a rare event. 

## 8.0 Contributing Factor Breakdown by Violator Risk


Analysis Ideas

1. Get % breakdown in a timeframe of all injury crashes. 
2. Show with 1, 2, 5, 10, 20 violation universe of cars all time, within 1y etc. 

Potentially: re-run with fault assumptions (no contributing factors on vehicle, not perfect)

In [ ]:
## TBD

# ANALYSIS 1: Chance of Incident After Xth Violation

This analysis looks at the year following a person's Xth violation and the chance, given the number of violations it is, they will get into an injury crash.

To Do: 

- Remove people who were killed during the study period in one of the crashes 

In [ ]:
db.dfquery("""

drop table if exists working.all_vehicle_crashes; 
create table working.all_vehicle_crashes as 

select distinct 

    --CRASH INFO 
    a.crashid, 
    a.crash_date,
    a.loc,
    
    --VEHICLE INFO 
    ve.vehicle_num, 
    replace(lower(ve.plate_num), ' ', '') plate_num, 
    
    --DRIVER LICENSE NUM (Refined)
    
    --AGE OF DRIVER BUCKET 
    case 
        when dd.victim_age is null or dd.victim_age < 15 or dd.victim_age > 110 then 'Unknown' 
        when dd.victim_age between 15 and 29 then 'Young'
        when dd.victim_age between 30 and 55 then 'Middle' 
        when dd.victim_age between 56 and 75 then 'Older' 
        when dd.victim_age between 76 and 110 then 'Very Old' 
    end as age,
    
    --SEX OF DRIVER 
    case 
        when dd.victim_sex is null then 'U'
        else dd.victim_sex 
    end as sex,
    
    --TLC PLATE INDICATOR 
    tlc_veh, 
    
    veh_type_general 
from 
    wc_accident_f a 

join 
    wc_accident_vehicle_f ve 

on 
    a.crashid=ve.crashid 

--GET DRIVER DEMO 
left join 
    working.driver_demo dd 
on 
    a.crashid=dd.crashid::numeric 
    and 
    replace(lower(ve.plate_num), ' ', '')=replace(lower(dd.plate_num), ' ', '')

--NO TLC 
join 
    working.tlc_plate_ind tlc 
on 
    lower(ve.plate_num)=lower(tlc.plate_num)

join 
    working.single_driver_plates pf 
on 
    replace(lower(ve.plate_num), ' ',  '')=pf.plate_num
    
--REMOVE JUNK PLATES 
join 
    working.cleaned_plates cp 
on 
    lower(ve.plate_num)=cp.plate_num 

where 
    --ONLY CRASHES 2017 ONWARDS 
    a.crash_date >= '2017-01-01'

    --ONLY CARE ABOUT NY PLATES 
    and lower(ve.plate_state)='ny' 

    --LIMIT TO CAR/MCL/BUS 
    and ve.veh_type_general in ('car/suv') --, 'truck/bus', 'mcl')

    --INJURY CRASHES 
    and a.crashid in (select crashid from wc_accident_victim_f)
   
    -- NO TLC VEHICLES 
    and not tlc_veh

order by 1; 

""")

Loaded Speed Camera Violations 
- Returns the violations with past violation info (violations in prior year, prior two years, etc.)
- Only limited to 2019 violations right now (for main violation, looks at huge universe for prior and after violations)

REUSE CODE FOR RLC



In [ ]:
db.dfquery("""

drop table if exists working.loaded_violations; 
create table working.loaded_violations as 

select distinct 
    v1.summons_number,
    v1.plate,
    v1.issue_date, 
    v1.violation_time, 
    v1.full_date,
    max((v2.full_date between v1.full_date and v1.full_date::date + 365)::int) = 0 as vio_within_12_months, 
    count(distinct case when v2.full_date < v1.full_date and v2.full_date >= v1.full_date::date - 365 then v2.summons_number end) as prev_1y_violations, 
    count(distinct case when v2.full_date < v1.full_date and v2.full_date >= v1.full_date::date - 365 then v2.summons_number end) as prev_2y_violations, 
    count(distinct case when v2.full_date < v1.full_date and v2.full_date >= v1.full_date::date - 365 then v2.summons_number end) as prev_3y_violations, 
    count(distinct case when v2.full_date < v1.full_date then v2.summons_number end) as prev_all_violations 
from 
    working.speed_camera_vio v1
left join
    working.speed_camera_vio v2
on 
    v1.plate=v2.plate
    and v1.summons_number != v2.summons_number
where
    v1.issue_date >= '2019-01-01' 
    and v1.issue_date <= '2019-12-31'
    and (v2.plate is null or v2.issue_date::date <= v1.issue_date::date + 365)
    --LOWER? 
    and v1.state='NY'
    and (v2.plate is null or v2.state='NY')
    and v1.license_type = 'PAS'
    and (v2.plate is null or v2.license_type = 'PAS')
    and v1.full_date is not null 
group by 
    1, 2, 3, 4, 5; --, 6, 7;

""")

## A1.3 Crash Loaded Violations 

- Returns the violation with associated crash info (within 12 months of violation)
- Limited to 2019 crashes right now
- DOES THAT DATE RANGE MAKE SENSE?

In [ ]:
db.dfquery("""

drop table if exists working.crash_loaded_violations; 
create table working.crash_loaded_violations as 

with  all_loaded_crashes as (

    select 
        lv.summons_number, lv.plate, array_agg(distinct ve.crashid) crashids 

    from 
        working.loaded_violations lv 

    join 
        working.all_vehicle_crashes ve 

    on 
        lower(lv.plate)=lower(ve.plate_num) 
        and ve.crash_date >= lv.full_date 
        and ve.crash_date <= lv.full_date::date + 365

    where 
        ve.crash_date >= '2019-01-01'
    
    group by 
        1, 2

)

select distinct 
    lv.*, crashids

from 
    working.loaded_violations lv 

join 
    all_loaded_crashes lc
on 
    lv.summons_number=lc.summons_number 
    and 
    lower(lv.plate)=lower(lc.plate)
;

""")

In [ ]:
db.dfquery("""

select 
    * 
from
    working.crash_loaded_violations
limit 10 

""")

## A1.4 Final Study 

Of those who received their xth violation within the past 12 mo, 24 mo within the study period, how many and what percent got into a crash? 


In [ ]:
study_df = db.dfquery("""

with study_period_violations as (

    select 
        summons_number, plate, full_date, prev_1y_violations, prev_1y_violations + 1 as violation_count 
    from 
        working.loaded_violations 
    where 
        issue_date between '2019-01-01' and '2019-12-31'
)

select 
    violation_count,
    crash_ocurred, 
    count(*)
from (
    select distinct 
        spv.*, 
        case when clv.summons_number is not null then True else False end as Crash_Ocurred 
    
    from 
        study_period_violations spv 
    
    left join 
        working.crash_loaded_violations clv 
    
    on 
        spv.summons_number=clv.summons_number 
) s  
group by 
    1, 2
order by 1 asc

""").pivot(index='violation_count', columns='crash_ocurred', values='count').reset_index()

study_df['%'] = study_df[True]/(study_df[True] + study_df[False])*100
study_df['%'] = study_df['%'].round(2)

study_df[study_df['violation_count'] <= 50].plot(x='violation_count', y='%') #.to_csv('Example.Csv')

In [ ]:
study_df[study_df[False] > 50]

## Another Study Design

Looks at the maximum number of violations within a certain timeframe (ex. 2019) and the chance within that timeframe that cohort got into a crash

In [ ]:
study_df = db.dfquery("""

with study_period_violations as (

    select 
        summons_number, plate, full_date, prev_1y_violations, prev_1y_violations + 1 as violation_count 
    from 
        working.loaded_violations 
    where 
        issue_date between '2019-01-01' and '2019-12-31'

)

select 
    violation_count, crash_ocurred, count(*) 
from (
    select 
        plate,
        max(violation_count) as violation_count,
        max(crash_ocurred::int)::boolean as crash_ocurred 
    from (
        select distinct 
            spv.*, 
            case when clv.summons_number is not null then True else False end as Crash_Ocurred 

        from 
            study_period_violations spv 

        left join 
            working.crash_loaded_violations clv 

        on 
            spv.summons_number=clv.summons_number 
    ) s  
    group by 
        1
) p
group by 1, 2

""").pivot(index='violation_count', columns='crash_ocurred', values='count').reset_index()

study_df['%'] = study_df[True]/(study_df[True] + study_df[False])*100
study_df['%'] = study_df['%'].round(2)

study_df[study_df['violation_count'] <= 30].plot(x='violation_count', y='%') #.to_csv('Example.Csv')

## Output

In [ ]:
writer.save()